# Sorting and fast-track alogorithms
By the end of this lecture you will be able to:
- sort a `DataFrame`
- sort a column with an expression 
- take advantage of fast-track algorithms with `set_sorted`

In this lecture we learn how to sort both on a `DataFrame` and within an expression. We also introduce the fast-track algorithms on sorted data. The fast-track algorithims are optimisations separate from those of the built-in query optimiser. We see how to take advantage of them here.

In [1]:
import polars as pl

Check out my short youtube video on this topic below

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/oRv1ANrW020?si=LbV4gQpaX--d2106" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

In [3]:
csv_file = "data_titanic.csv"

In [4]:
df = pl.read_csv(csv_file)
df.head(3)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S"""


We use `pl.Config` to adjust the default so that only 4 rows of a `DataFrame` are printed in this notebook

In [5]:
pl.Config.set_tbl_rows(4)

polars.config.Config

## Sorting a `DataFrame`

### Using the `sort` method on `DataFrame`

We can sort a `DataFrame` on a column with the `sort` method

In [6]:
df.sort("Age")

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
6,0,3,"""Moran, Mr. Jam…","""male""",null,0,0,"""330877""",8.4583,null,"""Q"""
18,1,2,"""Williams, Mr. …","""male""",null,0,0,"""244373""",13.0,null,"""S"""
…,…,…,…,…,…,…,…,…,…,…,…
852,0,3,"""Svensson, Mr. …","""male""",74.0,0,0,"""347060""",7.775,null,"""S"""
631,1,1,"""Barkworth, Mr.…","""male""",80.0,0,0,"""27042""",30.0,"""A23""","""S"""


By default `null` values are at the start of the sort. We can move the `nulls` to the end of the sort by setting the `nulls_last` argument to `True`

In [7]:
df.sort("Age",nulls_last=True)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
804,1,3,"""Thomas, Master…","""male""",0.42,0,1,"""2625""",8.5167,null,"""C"""
756,1,2,"""Hamalainen, Ma…","""male""",0.67,1,1,"""250649""",14.5,null,"""S"""
…,…,…,…,…,…,…,…,…,…,…,…
879,0,3,"""Laleff, Mr. Kr…","""male""",null,0,0,"""349217""",7.8958,null,"""S"""
889,0,3,"""Johnston, Miss…","""female""",null,1,2,"""W./C. 6607""",23.45,null,"""S"""


We can sort in reverse order with the `descending` argument - note that the `nulls_last` argument is set to the default of `True` so the `null` rows are first

In [8]:
df.sort("Age",descending=True)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
889,0,3,"""Johnston, Miss…","""female""",null,1,2,"""W./C. 6607""",23.45,null,"""S"""
879,0,3,"""Laleff, Mr. Kr…","""male""",null,0,0,"""349217""",7.8958,null,"""S"""
…,…,…,…,…,…,…,…,…,…,…,…
756,1,2,"""Hamalainen, Ma…","""male""",0.67,1,1,"""250649""",14.5,null,"""S"""
804,1,3,"""Thomas, Master…","""male""",0.42,0,1,"""2625""",8.5167,null,"""C"""


We get the largest values first by setting `nulls_last=True`

In [9]:
df.sort("Age",descending=True,nulls_last=True)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
631,1,1,"""Barkworth, Mr.…","""male""",80.0,0,0,"""27042""",30.0,"""A23""","""S"""
852,0,3,"""Svensson, Mr. …","""male""",74.0,0,0,"""347060""",7.775,null,"""S"""
…,…,…,…,…,…,…,…,…,…,…,…
18,1,2,"""Williams, Mr. …","""male""",null,0,0,"""244373""",13.0,null,"""S"""
6,0,3,"""Moran, Mr. Jam…","""male""",null,0,0,"""330877""",8.4583,null,"""Q"""


## Sort on multiple columns
We can sort based on multiple columns with either a list...

In [10]:
df.sort(["Pclass","Age"])

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
32,1,1,"""Spencer, Mrs. …","""female""",null,1,0,"""PC 17569""",146.5208,"""B78""","""C"""
56,1,1,"""Woolner, Mr. H…","""male""",null,0,0,"""19947""",35.5,"""C52""","""S"""
…,…,…,…,…,…,…,…,…,…,…,…
117,0,3,"""Connors, Mr. P…","""male""",70.5,0,0,"""370369""",7.75,null,"""Q"""
852,0,3,"""Svensson, Mr. …","""male""",74.0,0,0,"""347060""",7.775,null,"""S"""


...or with comma-separated strings

In [11]:
df.sort("Pclass","Age")

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
32,1,1,"""Spencer, Mrs. …","""female""",null,1,0,"""PC 17569""",146.5208,"""B78""","""C"""
56,1,1,"""Woolner, Mr. H…","""male""",null,0,0,"""19947""",35.5,"""C52""","""S"""
…,…,…,…,…,…,…,…,…,…,…,…
117,0,3,"""Connors, Mr. P…","""male""",70.5,0,0,"""370369""",7.75,null,"""Q"""
852,0,3,"""Svensson, Mr. …","""male""",74.0,0,0,"""347060""",7.775,null,"""S"""


## Sorting a column with an expression

We can transform a column into sorted order within an expression.

In this example we sort the values in every column independent of other columns

In [12]:
(
    df
    .select(
        pl.all().sort()
    )
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,1,"""Abbing, Mr. An…","""female""",null,0,0,"""110152""",0.0,null,null
2,0,1,"""Abbott, Mr. Ro…","""female""",null,0,0,"""110152""",0.0,null,null
…,…,…,…,…,…,…,…,…,…,…,…
890,1,3,"""van Billiard, …","""male""",74.0,8,5,"""WE/P 5735""",512.3292,"""G6""","""S"""
891,1,3,"""van Melkebeke,…","""male""",80.0,8,6,"""WE/P 5735""",512.3292,"""T""","""S"""


Within an expression we can also sort all columns with respect to another column using `sort_by`

In [13]:
(
    df
    .select(
        pl.all().sort_by("Age")
    )
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
6,0,3,"""Moran, Mr. Jam…","""male""",null,0,0,"""330877""",8.4583,null,"""Q"""
18,1,2,"""Williams, Mr. …","""male""",null,0,0,"""244373""",13.0,null,"""S"""
…,…,…,…,…,…,…,…,…,…,…,…
852,0,3,"""Svensson, Mr. …","""male""",74.0,0,0,"""347060""",7.775,null,"""S"""
631,1,1,"""Barkworth, Mr.…","""male""",80.0,0,0,"""27042""",30.0,"""A23""","""S"""


It seems like `sort_by` in this case has just replicated the functionality of 
```python
df.sort("Age")
```
However, as we can use `sort_by` in an expression it can be used in other contexts such as in a `groupby` aggregation.  For example, if we wanted to get the name and age of the oldest passenger in each class we can do the following

In [14]:
pl.Config.set_fmt_str_lengths(100)
(
    df
    .group_by("Pclass")
    .agg(
        pl.col("Name").sort_by("Age").last(),
        pl.col("Age").sort_by("Age").last()
        
    )
)

Pclass,Name,Age
i64,str,f64
2,"""Mitchell, Mr. Henry Michael""",70.0
1,"""Barkworth, Mr. Algernon Henry Wilson""",80.0
3,"""Svensson, Mr. Johan""",74.0


### Filtering for the largest/smallest values
If we just want to find the largest or smallest values we could do `sort` followed by `head` or `tail`. For example here we find the oldest passengers

In [15]:
(
    df
    .sort("Age")
    .tail(3)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
494,0,1,"""Artagaveytia, Mr. Ramon""","""male""",71.0,0,0,"""PC 17609""",49.5042,null,"""C"""
852,0,3,"""Svensson, Mr. Johan""","""male""",74.0,0,0,"""347060""",7.775,null,"""S"""
631,1,1,"""Barkworth, Mr. Algernon Henry Wilson""","""male""",80.0,0,0,"""27042""",30.0,"""A23""","""S"""


A faster approach is to use `top_k` which does not sort the full `DataFrame` but instead just searches through the rows to filter for the largest/smallest values and then sorts this small subset of rows

In [16]:
(
    df
    .top_k(
        # Number of records to return
        k=5,
        # Column/expression to sort by
        by="Age",
        # Return the largest records
        descending=False,
        # Ensure the nulls are at the end
        nulls_last=True
    )
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
631,1,1,"""Barkworth, Mr. Algernon Henry Wilson""","""male""",80.0,0,0,"""27042""",30.0,"""A23""","""S"""
852,0,3,"""Svensson, Mr. Johan""","""male""",74.0,0,0,"""347060""",7.775,null,"""S"""
…,…,…,…,…,…,…,…,…,…,…,…
97,0,1,"""Goldschmidt, Mr. George B""","""male""",71.0,0,0,"""PC 17754""",34.6542,"""A5""","""C"""
117,0,3,"""Connors, Mr. Patrick""","""male""",70.5,0,0,"""370369""",7.75,null,"""Q"""


Some good news: if you do .`sort.head/tail` in lazy mode Polars applies a `top_k` optimization under the hood

In [17]:
(
    df
    .lazy()
    .sort("Age")
    .tail(3)
    .collect()
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
494,0,1,"""Artagaveytia, Mr. Ramon""","""male""",71.0,0,0,"""PC 17609""",49.5042,null,"""C"""
852,0,3,"""Svensson, Mr. Johan""","""male""",74.0,0,0,"""347060""",7.775,null,"""S"""
631,1,1,"""Barkworth, Mr. Algernon Henry Wilson""","""male""",80.0,0,0,"""27042""",30.0,"""A23""","""S"""


## Taking advantage of sorted data

For some operations Polars can use a fast track algorithm if it knows the data in a column is sorted.

For example, if we want the `max` value on a sorted column a fast-track algorithm would just take the last (non-`null`) value.

See my blog post for more background on this: https://www.rhosignal.com/posts/polars-loves-sorted-data-1-statistics/

### Checking the sorted status
You can check if Polars **thinks** a column is sorted with the `flags` attribute on a column or a `Series`

In [22]:
df["PassengerId"].flags

{'SORTED_ASC': False, 'SORTED_DESC': False}

In this case as both the ASC and DESC values are `False` Polars doesn't think the `PassengerID` column is sorted (although we know that is sorted).

You can check the status of all columns at once with the `flags` attribute on a `DataFrame`

In [23]:
df.flags

{'PassengerId': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'Survived': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'Pclass': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'Name': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'Sex': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'Age': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'SibSp': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'Parch': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'Ticket': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'Fare': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'Cabin': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'Embarked': {'SORTED_ASC': False, 'SORTED_DESC': False}}

We can check if a column is actually sorted with the `is_sorted` method:

In [24]:
df["PassengerId"].is_sorted()

True

### Setting the sorted status
If we know that a column is sorted then we can let Polars know using `set_sorted`

In [26]:
df = (
    pl.read_csv(csv_file)
    .with_columns(
        pl.col("PassengerId").set_sorted()
    )
)
df["PassengerId"].flags

{'SORTED_ASC': True, 'SORTED_DESC': False}

Looking at the output of `flags` we now see `'SORTED_ASC': True`

In the exercises we see the major effect `set_sorted` can have on performance.

If we transform a column with a sorting operation Polars will automatically update the `flags` attribute for that column

In [27]:
df = (
    pl.read_csv(csv_file)
    .sort("PassengerId")
)
df["PassengerId"].flags

{'SORTED_ASC': True, 'SORTED_DESC': False}

If the data is sorted descending we tell Polars this by passing the `descending` argument:
```python
pl.col("PassengerId").set_sorted(descending=True)
```

### `set_sorted` in an expression
We can use `set_sorted` within an expression. 

For example, if we have a sorted column we can use `set_sorted` to find the `max`

In [28]:
(
    df
    .select(
        pl.col("PassengerId").set_sorted().max()
    )
)

PassengerId
i64
891


### Operations with fast-track algorithms
The set of operations that have sorted fast-track algorithms is evolving but includes:
- min
- max
- quantile
- median (a special case of quantile)
- filter
- group_by (see the groupby lectures)
- join (see the join lectures)

## Exercises
In the exercises you will develop your understanding of:
- sorting a `DataFrame`
- sorting in an expression
- using fast-track algorithms with `set_sorted`

## Exercise 1: Sorting a `DataFrame`
Sort the `DataFrame` by whether passengers survived and the alphabetical order of the passenger names.

Configure the output to print 10 lines using `pl.Config`

In [30]:
pl.Config<blank>
(
    pl.read_csv(csv_file)
)

SyntaxError: invalid syntax (798370332.py, line 1)

Add a column for the `familySize` which is the sum of the number of siblings (`SibSp` columns), the number of parents or children (`Parch` columns) plus one for the passenger themself.

Then sort all of the columns by `familySize` inside an expression

In [ ]:
(
    pl.read_csv(csv_file)
    <blank>
)

## Exercise 2: Using `set_sorted`

For this exercise we first create a random array in Numpy and then sort it.

We populate a `DataFrame` with the same array in 2 columns:
- a column that we **will** tell Polars is sorted called `known_sorted`
- a column that we **will not** tell Polars is sorted called `unknown_sorted`

In [34]:
# Create the sorted array
import numpy as np

N = 10_000_000
sorted_array = np.sort(np.random.standard_normal(N))
# Create the DataFrame
df_sort = pl.DataFrame({"known_sorted": sorted_array, "unknown_sorted": sorted_array})

df_sort.head(3)

known_sorted,unknown_sorted
f64,f64
-5.294323,-5.294323
-5.057108,-5.057108
-5.052246,-5.052246


Check to see if Polars thinks the `known_sorted` column is sorted yet

In [35]:
df_sort.flags

{'known_sorted': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'unknown_sorted': {'SORTED_ASC': False, 'SORTED_DESC': False}}

Tell polars that the `known_sorted` column is sorted

In [36]:
df_sort = (
    df_sort['known_sorted'].set_sorted()
)
df_sort.flags

{'SORTED_ASC': True, 'SORTED_DESC': False}

Confirm that Polars knows the `known_sorted` column is sorted

In [37]:
df_sort.flags

{'SORTED_ASC': True, 'SORTED_DESC': False}

Compare how long it takes to find the median of each column. 

> Ignore any you get a message saying that one run took much longer than the others and intermediate values might be cached. This variability in run time is just due to natural variability in runtime. Generally I run it again until i get a stable timing

In [40]:
%%timeit -n1 -r5
(
    df_sort['known_sorted'].median()
)

TypeError: cannot use `__getitem__` on Series of dtype Float64 with argument 'known_sorted' of type 'str'

In [ ]:
%%timeit -n1 -r5
(
    df_sort
    <blank>
)

We want to filter each `DataFrame` to find values less than -2. Compare how long it takes when we apply the filter to the `known_sorted` column compared to the `unknown_sorted` column

In [ ]:
%%timeit -n1 -r5
(
    df_sort
    <blank>
)

The size of the performance difference varies depending on where in the sorted range we are looking for values. To explore this you can comapare performance for e.g. >-2 or <4

You can also vary `N` to see how the difference changes with length.

## Solutions

## Solution to Exercise 1
Sort the `DataFrame` by survival and alphabetical order of the passenger names

In [29]:
pl.Config.set_tbl_rows(10)
(
    pl.read_csv(csv_file)
    .sort(
        ["Survived","Name"]
    )
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
846,0,3,"""Abbing, Mr. Anthony""","""male""",42.0,0,0,"""C.A. 5547""",7.55,null,"""S"""
747,0,3,"""Abbott, Mr. Rossmore Edward""","""male""",16.0,1,1,"""C.A. 2673""",20.25,null,"""S"""
309,0,2,"""Abelson, Mr. Samuel""","""male""",30.0,1,0,"""P/PP 3381""",24.0,null,"""C"""
366,0,3,"""Adahl, Mr. Mauritz Nils Martin""","""male""",30.0,0,0,"""C 7076""",7.25,null,"""S"""
402,0,3,"""Adams, Mr. John""","""male""",26.0,0,0,"""341826""",8.05,null,"""S"""
…,…,…,…,…,…,…,…,…,…,…,…
56,1,1,"""Woolner, Mr. Hugh""","""male""",null,0,0,"""19947""",35.5,"""C52""","""S"""
831,1,3,"""Yasbeck, Mrs. Antoni (Selini Alexander)""","""female""",15.0,1,0,"""2659""",14.4542,null,"""C"""
326,1,1,"""Young, Miss. Marie Grice""","""female""",36.0,0,0,"""PC 17760""",135.6333,"""C32""","""C"""


Add a column for the `familySize` which is the sum of the number of siblings (`SibSp` columns), the number of parents or children (`Parch` columns) plus one for the passenger themself.

Then sort all of the columns by `familySize` inside an expression

In [31]:
(
    pl.read_csv(csv_file)
    .with_columns( 
        (
            pl.col('SibSp') + pl.col('Parch') + 1
        ).alias('familySize')
    )
    .select(
        pl.all().sort_by("familySize")
    )
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,familySize
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str,i64
3,1,3,"""Heikkinen, Miss. Laina""","""female""",26.0,0,0,"""STON/O2. 3101282""",7.925,null,"""S""",1
5,0,3,"""Allen, Mr. William Henry""","""male""",35.0,0,0,"""373450""",8.05,null,"""S""",1
6,0,3,"""Moran, Mr. James""","""male""",null,0,0,"""330877""",8.4583,null,"""Q""",1
7,0,1,"""McCarthy, Mr. Timothy J""","""male""",54.0,0,0,"""17463""",51.8625,"""E46""","""S""",1
12,1,1,"""Bonnell, Miss. Elizabeth""","""female""",58.0,0,0,"""113783""",26.55,"""C103""","""S""",1
…,…,…,…,…,…,…,…,…,…,…,…,…
202,0,3,"""Sage, Mr. Frederick""","""male""",null,8,2,"""CA. 2343""",69.55,null,"""S""",11
325,0,3,"""Sage, Mr. George John Jr""","""male""",null,8,2,"""CA. 2343""",69.55,null,"""S""",11
793,0,3,"""Sage, Miss. Stella Anna""","""female""",null,8,2,"""CA. 2343""",69.55,null,"""S""",11


## Solution to exercise 2


Create the `DataFrame`

In [43]:
import numpy as np

N = 10_000_000
sorted_array = np.sort(np.random.standard_normal(N))
df_sort = pl.DataFrame({"known_sorted": sorted_array, "unknown_sorted": sorted_array})

df_sort.head(3)

known_sorted,unknown_sorted
f64,f64
-5.405904,-5.405904
-5.264049,-5.264049
-4.975963,-4.975963


Check to see if Polars thinks the `known_sorted` column is sorted yet

In [44]:
df_sort["known_sorted"].flags

{'SORTED_ASC': False, 'SORTED_DESC': False}

Tell polars that the `known_sorted` column is sorted

In [45]:
df_sort = (
    df_sort
    .with_columns(
        pl.col('known_sorted').set_sorted()
    )
)

Confirm that Polars knows the `known_sorted` column is sorted

In [46]:
df_sort["known_sorted"].flags

{'SORTED_ASC': True, 'SORTED_DESC': False}

Compare how long it takes to find the median of each column.  Ignore any you get a message saying that one run took much longer than the others and intermediate values might be cached. This variability in run time is just something that can happen

In [47]:
%%timeit -n1 -r5
(
    df_sort
    .select(
        pl.col("known_sorted").median()
    )
)

The slowest run took 6.63 times longer than the fastest. This could mean that an intermediate result is being cached.
309 µs ± 198 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [48]:
%%timeit -n1 -r5
(
    df_sort
    .select(
        pl.col("unknown_sorted").median()
    )
)

69.4 ms ± 14.6 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


We want to filter the `DataFrame` by the `known_sorted` and `unknown_sorted` columns to find values less than -2. Compare how long it takes when we apply the filter to the `known_sorted` column compared to the `unknown_sorted` column

In [49]:
%%timeit -n1 -r5
(
    df_sort
    .filter(
        pl.col("known_sorted") < -2
    )
)

3.47 ms ± 764 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [50]:
%%timeit -n1 -r5
(
    df_sort
    .filter(
        pl.col("unknown_sorted") < -2
    )
)

7.85 ms ± 1.01 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
